# Profundizando en Molecular Featurizations

In [5]:
import deepchem as dc
deepchem.__version__

'2.7.1'

Uno de los pasos más importantes para realizar el aprendizaje automático en datos moleculares es transformar los datos en una forma adecuada para la aplicación de algoritmos de aprendizaje. Este proceso se denomina ampliamente "featurización" e implica convertir una molécula en un vector o tensor de algún tipo. Existen varias formas diferentes de hacerlo, y la elección de la featurización a menudo depende del problema en cuestión. Ya hemos visto dos de estos métodos: las huellas moleculares y los objetos ConvMol para su uso con convoluciones en grafos. En este tutorial, examinaremos algunos de los otros métodos disponibles.


# Featurizers

En DC, un método de featurizar una molécula (o cualquier tipo de input) es definidir un **Featurizer Object**. Hay tres formas diferentes de usar los featurizers
* Cuando usamos los loaders de MoleculeNet, pasamos el nombre de featurización al método. Por ejemplo, **featurizer = 'ECFP'**
* Puedes crear un Featurizer y aplicarlo directamente a moléculas. Se verá un ejemplo en el código siguiente
* Cuando creas un nuevo dataset con el DataLoader framework, puedes especificar un Featurizer para procesar los datos.

In [6]:
'''
En este código lo que hacemos es crear un objeto de tipo 
Featurizer y aplicarselo a distintas moléculas representadas
mediante SMILES string
'''

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC', 'CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


En los siguientes ejemplos, usaremos el propano (CH3CH2CH3, representado por SMILES string como 'CCC')

Algunas técnicas de featurización utilizan información de múltiples conformaciones de una molécula para capturar de manera más completa su comportamiento químico y físico. Esto es particularmente importante para compuestos que pueden cambiar de forma o tener múltiples estados conformacionales. Esto es lo que se le conoce como "conformers" o conformaciones. 

Una conformación puede ser generada usando la clase en `ConformerGenerator` en `deepchem.utils.conformers`

## RDKitDescriptors

**RDKitDescriptors** featuriza una molécula utilizando RDKit para calcular **valores** para una **lista de descriptores.** Estos **descriptores** son **propiedades físicas y químicas básicas**, como el **peso molecular, el área superficial polar, el número de donantes y aceptores de enlaces de hidrógeno,** entre otros. Esto resulta más **útil** para **predecir cosas** que **dependen** de **estas propiedades de alto nivel en lugar de la estructura molecular detallada.**

Inherente al featurizador se encuentra un conjunto de descriptores permitidos, que se pueden acceder mediante **RDKitDescriptors.allowedDescriptors**. El featurizador utiliza los descriptores en **rdkit.Chem.Descriptors.descList**, verifica si están en la lista de descriptores permitidos y calcula el valor del descriptor para la molécula.

A continuación, imprimiremos los valores de los primeros diez descriptores para el propano.


In [14]:
rdkit_featurizer = dc.feat.RDKitDescriptors() #Creamos el featurizer
features = rdkit_featurizer(['CCC'])[0] #Lo aplicamos al propano

#Imprimimos las 10 primeras carácterísticas y su valor
for feature, descriptor in zip(features[:10], rdkit_featurizer.descriptors):
    print(descriptor, feature)

MaxAbsEStateIndex 2.125
MaxEStateIndex 2.125
MinAbsEStateIndex 1.25
MinEStateIndex 1.25
qed 0.3854706587740357
MolWt 44.097
HeavyAtomMolWt 36.033
ExactMolWt 44.062600255999996
NumValenceElectrons 20.0
NumRadicalElectrons 0.0


209

In [16]:
#Número total de descriptores del propano
print('The number of descriptors present is: ', len(features))

The number of descriptors present is:  209


## WeaveFeaturizer and MolGraphConvFeaturizer

Anteriormente, examinamos las convoluciones en grafos, que utilizan **ConvMolFeaturizer** para convertir moléculas en objetos ConvMol. Las convoluciones en grafos son un caso especial de una amplia clase de arquitecturas que representan las moléculas como grafos. Funcionan de manera similar pero varían en los detalles. Por ejemplo, pueden asociar vectores de datos con los átomos, los enlaces que los conectan o ambos. Pueden utilizar una variedad de técnicas para calcular nuevos vectores de datos a partir de los de la capa anterior y una variedad de técnicas para calcular propiedades a nivel de molécula al final.

DeepChem admite muchos modelos basados en grafos diferentes. Algunos de ellos requieren que las moléculas se featuricen de manera ligeramente diferente. Debido a esto, existen otros dos featurizers llamados **WeaveFeaturizer** y **MolGraphConvFeaturizer**. Cada uno de ellos convierte las moléculas en un tipo diferente de objeto Python que se utiliza en modelos particulares. Al usar cualquier modelo basado en grafos, simplemente consulta la documentación para ver qué featurizer debes utilizar con él.


## CoulombMatrix

Hasta ahora, todos los modelos que hemos analizado consideran solo las propiedades intrínsecas de una molécula: la lista de átomos que la componen y los enlaces que los conectan. Al trabajar con moléculas flexibles, es posible que también desees tener en cuenta las diferentes conformaciones que la molécula puede asumir. Por ejemplo, cuando una molécula de fármaco se une a una proteína, la fuerza de la unión depende de interacciones específicas entre pares de átomos. Para **predecir la fuerza de unión, probablemente desees considerar una variedad de posibles conformaciones y utilizar un modelo que las tenga en cuenta al realizar predicciones.**

La **matriz de Coulomb** es una popular featurización para conformaciones moleculares. Recordemos que la interacción electrostática de Coulomb entre dos cargas es proporcional a  
q1q2/r donde q1 y q2 son las cargas y r es la distancia entre ellas. Para una molécula con N átomos, la matriz de Coulomb es una matriz de N×N en la que cada elemento **proporciona la intensidad de la interacción electrostática entre dos átomos.** Contiene información tanto sobre las cargas en los átomos como sobre las distancias entre ellos. Puedes encontrar más información sobre las formas funcionales utilizadas https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.108.058301

Para aplicar este featurizador, primero necesitamos un conjunto de conformaciones para la molécula. Podemos utilizar la clase **ConformerGenerator** para hacerlo. Toma una molécula RDKit, genera un conjunto de conformaciones minimizadas en energía y elimina aquellas que sean significativamente diferentes entre sí. Vamos a intentar ejecutarlo para el propano.


In [17]:
from rdkit import Chem

#Creamos en generador de conformaciones
generator = dc.utils.ConformerGenerator(max_conformers=5)
#Obtenemos las primeras 5 conformaciones aplicando el generador al propano
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
#Imprimimos el número de conformaciones disponibles
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

Number of available conformers for propane:  1


Solo encuentra una conformación disponible debido  ala simplicidad del propano. Es una molécula pequeña con poca flexibilidad. Probemos añadiendo otro carbono (el butano)

In [18]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


Creemos ahora la Coulomb Matrix para la molécula

In [88]:
'''
Esto lo que nos devuelve es por cada átomo la interacción electrostática
entre el resto de átomos que componen la molécula.
Si accedemos a features[0][2] obtendremos las interacciones electrostáticas
del segundo átomo con el resto de átomos
'''
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=11)
features = coulomb_mat(propane_mol)
print(features)
print(" ")
print("Interacciones electrostáticas con el átomo 2")
print(features[0][2])



[[[36.8581052  12.48684429  7.5619687   2.85945193  2.85804514
    2.85804556  1.4674015   1.46740144  0.91279491  1.14239698
    1.14239675]
  [12.48684429 36.8581052  12.48684388  1.46551218  1.45850736
    1.45850732  2.85689525  2.85689538  1.4655122   1.4585072
    1.4585072 ]
  [ 7.5619687  12.48684388 36.8581052   0.9127949   1.14239695
    1.14239692  1.46740146  1.46740145  2.85945178  2.85804504
    2.85804493]
  [ 2.85945193  1.46551218  0.9127949   0.5         0.29325367
    0.29325369  0.21256978  0.21256978  0.12268391  0.13960187
    0.13960185]
  [ 2.85804514  1.45850736  1.14239695  0.29325367  0.5
    0.29200271  0.17113413  0.21092513  0.13960186  0.1680002
    0.20540029]
  [ 2.85804556  1.45850732  1.14239692  0.29325369  0.29200271
    0.5         0.21092513  0.17113413  0.13960187  0.20540032
    0.16800016]
  [ 1.4674015   2.85689525  1.46740146  0.21256978  0.17113413
    0.21092513  0.5         0.29351308  0.21256981  0.2109251
    0.17113412]
  [ 1.46740144  

Observa que muchos elementos son 0. Para combinar múltiples moléculas en un lote, necesitamos que todas las matrices de Coulomb tengan el mismo tamaño, incluso si las moléculas tienen diferentes números de átomos. Especificamos `max_atoms=20`, por lo que la matriz devuelta tiene un tamaño de (20, 20). Sin embargo, la molécula solo tiene 11 átomos, por lo que solo una submatriz de 11 por 11 no es nula.


## CoulombMatrixEig

Una característica importante de las **matrices de Coulomb** es que **son invariantes a la rotación y la traslación molecular**, ya que las distancias interatómicas y los números atómicos no cambian. **Respetar simetrías como esta facilita el aprendizaje.** Rotar una molécula no cambia sus propiedades físicas. Si la featurización cambia con la rotación, entonces el modelo debe aprender que las rotaciones no son importantes, pero si la featurización es invariante, el modelo adquiere esta propiedad automáticamente.

Sin embargo, **las matrices de Coulomb no son invariantes bajo** otra simetría importante: **permutaciones de los índices de los átomos.** Las **propiedades físicas de una molécula no dependen de cuál llamemos "átomo 1", pero la matriz de Coulomb sí lo hace.** Para abordar esto, se introdujo el featurizador **CoulombMatrixEig**, que utiliza el espectro de autovalores de la matriz de Coulomb y es invariante a permutaciones aleatorias de los índices de los átomos. La desventaja de esta featurización es que contiene mucha menos información (autovalores en lugar de una matriz  
N
×
N
 ), por lo que los modelos estarán más limitados en lo que pueden aprender.

**CoulombMatrixEig** hereda de **CoulombMatrix** y realiza la featurización de una molécula calculando primero las matrices de Coulomb para diferentes conformaciones de la molécula y luego calculando los autovalores para cada matriz de Coulomb. Estos autovalores se ajustan para tener en cuenta la variación en el número de átomos entre las moléculas.


In [89]:
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=11)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620303 29.62963149 22.75497781  0.5713786   0.28781332  0.28548338
   0.27558187  0.18163794  0.17460999  0.17059719  0.16640098]]
